In [155]:
# !rm -rf /content/Instance_level_recognition

In [156]:
# ATTENTION: execute more than once create non-needed directory owing to current directory change
!git clone https://github.com/bird0401/Instance_level_recognition.git

Cloning into 'Instance_level_recognition'...
remote: Enumerating objects: 736, done.
remote: Counting objects: 100% (319/319), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 736 (delta 165), reused 236 (delta 103), pack-reused 417
Receiving objects: 100% (736/736), 25.89 MiB | 18.43 MiB/s, done.
Resolving deltas: 100% (348/348), done.


In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import joblib

# import image dataset from GCS

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# ! echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" |sudo tee /etc/apt/sources.list.d/gcsfuse.list
# ! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
# ! apt-get -y -q update
# ! apt-get -y -q install gcsfuse

In [ ]:
# ! mkdir -p ./data
# ! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 entity_dogs ./data

# Create csv

In [ ]:
# # get file paths
# import glob
# l = glob.glob('/content/Instance_level_recognition/app/ml/data/imgs/**/*')

In [ ]:
# df = pd.DataFrame(l, columns=["path"])
# df.head()

In [ ]:
# def get_wikidata_id(path):
#   return path.split("/")[-2]

In [ ]:
# df['wikidata_id'] = df['path'].apply(get_wikidata_id)
# df.head()

# Extract images with more than or equal to 3 images

In [162]:
# df = pd.read_csv("/content/Instance_level_recognition/app/ml/data/path_label.csv", index_col=0)
# df

,path,wikidata_id,label
0,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
1,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
2,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
3,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
4,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
...,...,...,...
23040,/content/Instance_level_recognition/app/ml/dat...,Q860671,480
23041,/content/Instance_level_recognition/app/ml/dat...,Q860671,480
23042,/content/Instance_level_recognition/app/ml/dat...,Q860671,480
23043,/content/Instance_level_recognition/app/ml/dat...,Q860671,480


In [163]:
vc = df['wikidata_id'].value_counts()
used_wikidata_id = vc[vc >= 5].index
df = df[df["wikidata_id"].isin(used_wikidata_id)]
df = df.reset_index(drop = True)
df

,path,wikidata_id,label
0,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
1,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
2,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
3,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
4,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
...,...,...,...
22862,/content/Instance_level_recognition/app/ml/dat...,Q860671,480
22863,/content/Instance_level_recognition/app/ml/dat...,Q860671,480
22864,/content/Instance_level_recognition/app/ml/dat...,Q860671,480
22865,/content/Instance_level_recognition/app/ml/dat...,Q860671,480


In [164]:
def Encoding(df, filename):
  encoder = LabelEncoder()
  df['label'] = encoder.fit_transform(df['wikidata_id'])

  with open(f"{filename}.pkl", "wb") as fp:
      joblib.dump(encoder, fp)

  return df

In [165]:
df = Encoding(df, "le")
df

,path,wikidata_id,label
0,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
1,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
2,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
3,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
4,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0
...,...,...,...
22862,/content/Instance_level_recognition/app/ml/dat...,Q860671,447
22863,/content/Instance_level_recognition/app/ml/dat...,Q860671,447
22864,/content/Instance_level_recognition/app/ml/dat...,Q860671,447
22865,/content/Instance_level_recognition/app/ml/dat...,Q860671,447


In [166]:
df.to_csv("/content/Instance_level_recognition/app/ml/data/path_label.csv", index=False)

# Split csv to train and test

In [169]:
def Split_Kfold(df, n_fold):
  train_val_indices, test_indices = train_test_split(list(range(len(df.label))), test_size=0.2, stratify=df.label)
  df.loc[train_val_indices, "is_train_val"] = 1
  df.loc[test_indices, "is_train_val"] = 0
  df_train, df_test = df[df["is_train_val"] == 1].reset_index(drop = True), df[df["is_train_val"] == 0].reset_index(drop = True)
  print(len(df_train), len(df_test))

  skf = StratifiedKFold(n_splits=n_fold)
  for fold, ( _, val_) in enumerate(skf.split(X=df_train, y=df_train.label)):
        df_train.loc[val_ , "kfold"] = fold
  display(df_train)
  return df_train, df_test

In [170]:
df_train, df_test = Split_Kfold(df, 5)

18293 4574


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


,path,wikidata_id,label,is_train_val,kfold
0,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0,1.0,0.0
1,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0,1.0,0.0
2,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0,1.0,0.0
3,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0,1.0,0.0
4,/content/Instance_level_recognition/app/ml/dat...,Q10345998,0,1.0,0.0
...,...,...,...,...,...
18288,/content/Instance_level_recognition/app/ml/dat...,Q860671,447,1.0,4.0
18289,/content/Instance_level_recognition/app/ml/dat...,Q860671,447,1.0,4.0
18290,/content/Instance_level_recognition/app/ml/dat...,Q860671,447,1.0,4.0
18291,/content/Instance_level_recognition/app/ml/dat...,Q860671,447,1.0,4.0


In [171]:
df_train.to_csv("/content/Instance_level_recognition/app/ml/data/train.csv")
df_test.to_csv("/content/Instance_level_recognition/app/ml/data/test.csv")

# Create csv of small data

In [173]:
# extract 10 classes
df = pd.read_csv("/content/Instance_level_recognition/app/ml/data/path_label.csv")
vc = df['wikidata_id'].value_counts()
used_wikidata_id = vc.sample(n=10, random_state=2022).index
df = df[df["wikidata_id"].isin(used_wikidata_id)]
df = df.reset_index(drop = True)
df

,path,wikidata_id,label
0,/content/Instance_level_recognition/app/ml/dat...,Q11716842,11
1,/content/Instance_level_recognition/app/ml/dat...,Q11716842,11
2,/content/Instance_level_recognition/app/ml/dat...,Q11716842,11
3,/content/Instance_level_recognition/app/ml/dat...,Q11716842,11
4,/content/Instance_level_recognition/app/ml/dat...,Q11716842,11
...,...,...,...
641,/content/Instance_level_recognition/app/ml/dat...,Q39280,412
642,/content/Instance_level_recognition/app/ml/dat...,Q39280,412
643,/content/Instance_level_recognition/app/ml/dat...,Q39280,412
644,/content/Instance_level_recognition/app/ml/dat...,Q39280,412


In [174]:
df = Encoding(df, "le_debug")
df

,path,wikidata_id,label
0,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0
1,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0
2,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0
3,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0
4,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0
...,...,...,...
641,/content/Instance_level_recognition/app/ml/dat...,Q39280,9
642,/content/Instance_level_recognition/app/ml/dat...,Q39280,9
643,/content/Instance_level_recognition/app/ml/dat...,Q39280,9
644,/content/Instance_level_recognition/app/ml/dat...,Q39280,9


In [175]:
df.to_csv("/content/Instance_level_recognition/app/ml/data/path_label_debug.csv")

In [176]:
df_train, df_test = Split_Kfold(df, 5)

516 130


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


,path,wikidata_id,label,is_train_val,kfold
0,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0,1.0,0.0
1,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0,1.0,0.0
2,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0,1.0,0.0
3,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0,1.0,0.0
4,/content/Instance_level_recognition/app/ml/dat...,Q11716842,0,1.0,0.0
...,...,...,...,...,...
511,/content/Instance_level_recognition/app/ml/dat...,Q39280,9,1.0,3.0
512,/content/Instance_level_recognition/app/ml/dat...,Q39280,9,1.0,3.0
513,/content/Instance_level_recognition/app/ml/dat...,Q39280,9,1.0,4.0
514,/content/Instance_level_recognition/app/ml/dat...,Q39280,9,1.0,4.0


In [177]:
df_train.to_csv("/content/Instance_level_recognition/app/ml/data/train_debug.csv")
df_test.to_csv("/content/Instance_level_recognition/app/ml/data/test_debug.csv")